In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
import cv2

In [ ]:
folder = 'track_1'

In [ ]:
main_folder = '../data/{}'.format(folder)
data = pd.read_csv('{}/driving_log_combined.csv'.format(main_folder))


In [ ]:
image_paths = []
measurements = []
for index, row in data.iterrows():
    img_name = row['center'].split('/')[-1]
    img_path = '{}/{}/IMG/{}'.format(main_folder, row['run'], img_name)
    image_paths.append(img_path)
    steering = row['steering']    
    measurements.append(steering)b

In [ ]:
from sklearn.model_selection import train_test_splitb

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(image_paths, measurements, test_size=0.2, random_state=42)

In [ ]:
def load_image(path):
    return cv2.imread(path)
def augment_image(img, steering):
    if np.random.rand() > 0.5:
        new_img = cv2.flip(img, 1)
        steering = steering * -1.0
        return new_img, steering
    return img, steering
def data_generator(is_train, batch_size, X, y):
    images = []
    measurements = []
    for index in np.random.randint(0,len(X), batch_size):
        path = X[index]
        steering = y[index]
        img = load_image(path)
        if is_train:
            img, steering = augment_image(img, steering)
        images.append(img)
        measurements.append(steering)

    images = np.array(images)
    measurements = np.array(measurements)    
    yield images, measurements

In [ ]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
from sklearn.model_selection import train_test_split #to split out training and testing data 
#The Sequential container is a linear stack of layers
from keras.models import Sequential
#popular optimization strategy that uses gradient descent 
from keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from keras.callbacks import ModelCheckpoint
#what types of layers do we want our model to have?
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#helper class to define input shape and generate training images given image paths & steering angles
#for command line arguments
import argparse
#for reading files
import os


In [ ]:
model = Sequential()
#model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(160,320,3)))
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2), input_shape=(160,320,3)))
#model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
#model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
#model.add(Conv2D(64, 3, 3, activation='elu'))
#model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')
b

In [ ]:
model.fit_generator(data_generator(is_train=True, batch_size=50, X=X_train, y=y_train),
                    10, # samples per epoch
                    10, # nb_epoch
                    max_q_size=1,
                    validation_data=data_generator(is_train=False, batch_size=50, X=X_valid, y=y_valid),
                    nb_val_samples=100)


In [ ]:
model_name = '../models/model_v{}.h5'.format(i)
model.save(model_name)
i = i+1
print(model_name)